In [0]:
pip install beautifulsoup4

In [0]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.linkedin.com/jobs/search/?currentJobId=4253000271&keywords=analista%20de%20dados&origin=BLENDED_SEARCH_RESULT_NAVIGATION_JOB_CARD&originToLandingJobPostings=4253000271%2C4269426730%2C4272758138"

try:
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Remove scripts e estilos para focar no texto visível
    for script in soup(["script", "style"]):
        script.decompose()
    
    texto = soup.get_text()
    
    # Busca por "analista de dados" (case-insensitive)
    padrao = re.compile(r'analista de dados', re.IGNORECASE)
    encontrados = padrao.finditer(texto)
    
    resultados = []
    for match in encontrados:
        inicio = max(0, match.start() - 100)
        fim = min(len(texto), match.end() + 100)
        contexto = texto[inicio:fim]
        resultados.append({
            'palavra': match.group(),
            'contexto': contexto,
            'posicao': match.start()
        })
    
    if resultados:
        print(f"Encontrado {len(resultados)} vez(es):")
        for r in resultados:
            print(f"- {r['contexto']}")
    else:
        print("Não foi encontrado 'analista de dados' na página.")
        
except Exception as e:
    print(f"Erro ao acessar a página: {e}")

In [0]:
import requests
from bs4 import BeautifulSoup
import re

# Lista de URLs para verificar (pode ser uma única URL)
urls = [
    "https://www.linkedin.com/jobs/search/?currentJobId=4253000271&keywords=analista%20de%20dados&origin=BLENDED_SEARCH_RESULT_NAVIGATION_JOB_CARD&originToLandingJobPostings=4253000271%2C4269426730%2C4272758138",
    # Adicione mais URLs aqui, por exemplo:
    # "https://vagas.empregosbrasil.com/analista-de-dados",
    # "https://www.linkedin.com/jobs/view/analista-de-dados-vaga/"
]

# Palavra-chave exata a buscar (minúsculas para evitar sensibilidade)
palavra_chave = "analista de dados"

# Padrao regex para busca exata (case-insensitive)
padrao = re.compile(re.escape(palavra_chave), re.IGNORECASE)

# Lista para armazenar resultados
resultados = []

for url in urls:
    try:
        print(f"\n🔍 Verificando: {url}")
        
        # Faz a requisição
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Analisa o HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove scripts e estilos para focar no texto visível
        for script in soup(["script", "style"]):
            script.decompose()

        # Extrai apenas o texto visível
        texto = soup.get_text(strip=True)

        # Remove espaços extras e quebras de linha
        texto = re.sub(r'\s+', ' ', texto)

        # Busca a palavra-chave exata
        if padrao.search(texto):
            resultados.append({
                'url': url,
                'encontrado': True,
                'contexto': None  # Pode adicionar contexto se necessário
            })
            print(f"✅ Encontrado em: {url}")
        else:
            resultados.append({
                'url': url,
                'encontrado': False,
                'contexto': None
            })
            print(f"❌ Não encontrado em: {url}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro ao acessar {url}: {e}")
        resultados.append({
            'url': url,
            'encontrado': False,
            'erro': str(e)
        })

# Resumo final
print("\n" + "="*50)
print("📊 RESUMO DOS RESULTADOS:")
print("="*50)

total_encontrados = sum(1 for r in resultados if r['encontrado'])
print(f"Total de URLs verificadas: {len(resultados)}")
print(f"Encontrado em: {total_encontrados} URL(s)")

if total_encontrados > 0:
    print("\n📌 URLs onde foi encontrado 'analista de dados':")
    for r in resultados:
        if r['encontrado']:
            print(f"  - {r['url']}")
else:
    print("\n❌ Nenhuma ocorrência encontrada em nenhuma URL.")